In [2]:
# ===============================
# 🧠 GRU-Based Insulin Recommender
# ===============================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import recall_score, precision_score, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import random

# ------------------------------
# ⚙️ 1. Read and preprocess data
# ------------------------------
def read_csv_data(filepath):
    df = pd.read_csv(filepath, sep=';')
    df.columns = df.columns.str.strip()

    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time').reset_index(drop=True)

    print(f"Data range: {df['time'].min()} → {df['time'].max()}")
    return df

# Replace path with your file
df = read_csv_data("HUPA0001P.csv")

# ------------------------------
# 🔢 2. Feature scaling
# ------------------------------
feature_cols = ['glucose', 'calories', 'heart_rate', 'steps', 'carb_input']
target_cols = ['basal_rate', 'bolus_volume_delivered']

# Scale inputs
scaler_X = MinMaxScaler()
df[feature_cols] = scaler_X.fit_transform(df[feature_cols])

# Scale basal normally
scaler_basal = MinMaxScaler()
df['basal_rate'] = scaler_basal.fit_transform(df[['basal_rate']])

# Scale bolus only on nonzero values (avoid all squashed zeros)
scaler_bolus = RobustScaler()
nonzero = df['bolus_volume_delivered'] > 0
if nonzero.any():
    df.loc[nonzero, 'bolus_volume_delivered'] = scaler_bolus.fit_transform(df.loc[nonzero, ['bolus_volume_delivered']])
else:
    df['bolus_volume_delivered'] = 0.0

# ------------------------------
# 🪄 3. Sequence creation
# ------------------------------
sequence_length = 10

def create_sequences(data, seq_len=10):
    X, y_basal, y_bolus, y_detect = [], [], [], []
    for i in range(len(data) - seq_len):
        seq_x = data[feature_cols].iloc[i:i+seq_len].values
        seq_y_basal = data['basal_rate'].iloc[i+seq_len]
        seq_y_bolus = data['bolus_volume_delivered'].iloc[i+seq_len]
        # label 1 if any bolus in window
        detect_label = 1.0 if data['bolus_volume_delivered'].iloc[i:i+seq_len].max() > 0 else 0.0

        X.append(seq_x)
        y_basal.append(seq_y_basal)
        y_bolus.append(seq_y_bolus)
        y_detect.append(detect_label)
    return np.array(X), np.array(y_basal), np.array(y_bolus), np.array(y_detect)

X, y_basal, y_bolus, y_detect = create_sequences(df, sequence_length)
print(f"Created {len(X)} sequences")

# ------------------------------
# ⚖️ 4. PROPER Train-Validation-Test Split
# ------------------------------

# First split: 80% train+val, 20% test
X_temp, X_test, yb_temp, yb_test, yob_temp, yob_test, yd_temp, yd_test = train_test_split(
    X, y_basal, y_bolus, y_detect, test_size=0.2, random_state=42, stratify=y_detect
)

# Second split: 80% of temp for train, 20% for validation (64% train, 16% val, 20% test)
X_train, X_val, yb_train, yb_val, yob_train, yob_val, yd_train, yd_val = train_test_split(
    X_temp, yb_temp, yob_temp, yd_temp, test_size=0.2, random_state=42, stratify=yd_temp
)

print(f"Training: {len(X_train)} sequences")
print(f"Validation: {len(X_val)} sequences")
print(f"Test: {len(X_test)} sequences")

# Convert to tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)

yb_train_t = torch.tensor(yb_train, dtype=torch.float32).unsqueeze(-1)
yob_train_t = torch.tensor(yob_train, dtype=torch.float32).unsqueeze(-1)
yd_train_t = torch.tensor(yd_train, dtype=torch.float32).unsqueeze(-1)

yb_val_t = torch.tensor(yb_val, dtype=torch.float32).unsqueeze(-1)
yob_val_t = torch.tensor(yob_val, dtype=torch.float32).unsqueeze(-1)
yd_val_t = torch.tensor(yd_val, dtype=torch.float32).unsqueeze(-1)

yb_test_t = torch.tensor(yb_test, dtype=torch.float32).unsqueeze(-1)
yob_test_t = torch.tensor(yob_test, dtype=torch.float32).unsqueeze(-1)
yd_test_t = torch.tensor(yd_test, dtype=torch.float32).unsqueeze(-1)

# Create datasets and loaders
train_ds = TensorDataset(X_train_t, yb_train_t, yob_train_t, yd_train_t)
val_ds = TensorDataset(X_val_t, yb_val_t, yob_val_t, yd_val_t)
test_ds = TensorDataset(X_test_t, yb_test_t, yob_test_t, yd_test_t)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

# ------------------------------
# 🧩 5. GRU model
# ------------------------------
class InsulinGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, dropout=0.2):
        super(InsulinGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True, dropout=dropout)
        self.fc_basal = nn.Linear(hidden_size, 1)
        self.fc_bolus = nn.Linear(hidden_size, 1)
        self.fc_detect = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        _, h = self.gru(x)
        h = self.dropout(h[-1])
        return self.fc_basal(h), self.fc_bolus(h), self.sigmoid(self.fc_detect(h))

model = InsulinGRU(len(feature_cols))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_reg = nn.MSELoss()
loss_cls = nn.BCELoss(weight=torch.tensor([3.0]))  # weight for imbalance

# ------------------------------
# 🎯 6. IMPROVED Training with Validation & Early Stopping
# ------------------------------
EPOCHS = 100
best_val_f1 = 0
patience = 15
patience_counter = 0
training_history = []

print("Starting training with validation...")
for epoch in range(1, EPOCHS+1):
    # Training phase
    model.train()
    total_loss = 0
    for Xb, yb, yob, yd in train_loader:
        optimizer.zero_grad()
        pred_basal, pred_bolus, pred_detect = model(Xb)
        loss = (
            loss_reg(pred_basal, yb) +
            loss_reg(pred_bolus, yob) +
            loss_cls(pred_detect, yd)
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation phase every epoch
    model.eval()
    val_preds, val_trues = [], []
    val_basal_preds, val_basal_trues = [], []
    val_bolus_preds, val_bolus_trues = [], []

    with torch.no_grad():
        for Xb, yb, yob, yd in val_loader:
            pb, po, pd = model(Xb)
            val_preds.extend(pd.cpu().numpy().flatten())
            val_trues.extend(yd.cpu().numpy().flatten())
            val_basal_preds.extend(pb.cpu().numpy().flatten())
            val_basal_trues.extend(yb.cpu().numpy().flatten())
            val_bolus_preds.extend(po.cpu().numpy().flatten())
            val_bolus_trues.extend(yob.cpu().numpy().flatten())

    val_preds = np.array(val_preds)
    val_trues = np.array(val_trues)
    val_pred_label = (val_preds > 0.5).astype(int)

    val_recall = recall_score(val_trues, val_pred_label, zero_division=0)
    val_precision = precision_score(val_trues, val_pred_label, zero_division=0)
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall + 1e-9)
    val_basal_mae = mean_absolute_error(val_basal_trues, val_basal_preds)

    # Store history
    training_history.append({
        'epoch': epoch,
        'train_loss': total_loss/len(train_loader),
        'val_f1': val_f1,
        'val_recall': val_recall,
        'val_precision': val_precision,
        'val_basal_mae': val_basal_mae
    })

    # Print progress every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{EPOCHS} - Train Loss: {total_loss/len(train_loader):.4f} | "
              f"Val F1: {val_f1:.3f} | Val Recall: {val_recall:.3f} | Val Precision: {val_precision:.3f}")

    # Early stopping check
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        best_epoch = epoch
    else:
        patience_counter += 1

    if patience_counter >= patience and epoch > 30:
        print(f" Early stopping at epoch {epoch}")
        print(f" Best validation F1: {best_val_f1:.3f} at epoch {best_epoch}")
        break

# Load best model for final evaluation
print("Loading best model for testing...")
model.load_state_dict(torch.load('best_model.pth'))

# ------------------------------
# 📊 7. COMPREHENSIVE Evaluation on Test Set
# ------------------------------
model.eval()
y_true, y_pred, y_true_bolus, y_pred_bolus, y_true_basal, y_pred_basal = [], [], [], [], [], []

with torch.no_grad():
    for Xb, yb, yob, yd in test_loader:
        pb, po, pd = model(Xb)
        y_pred.extend(pd.detach().cpu().numpy().flatten())
        y_true.extend(yd.detach().cpu().numpy().flatten())
        y_pred_bolus.extend(po.detach().cpu().numpy().flatten())
        y_true_bolus.extend(yob.detach().cpu().numpy().flatten())
        y_pred_basal.extend(pb.detach().cpu().numpy().flatten())
        y_true_basal.extend(yb.detach().cpu().numpy().flatten())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Find optimal threshold on validation set
print("\n Finding optimal threshold...")
best_threshold = 0.5
best_f1 = 0

for threshold in np.arange(0.3, 0.7, 0.02):
    y_pred_label = (y_pred > threshold).astype(int)
    recall = recall_score(y_true, y_pred_label, zero_division=0)
    precision = precision_score(y_true, y_pred_label, zero_division=0)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f" Optimal threshold: {best_threshold:.2f}")

# Final evaluation with optimal threshold
y_pred_label = (y_pred > best_threshold).astype(int)

recall = recall_score(y_true, y_pred_label)
precision = precision_score(y_true, y_pred_label)
f1 = 2 * (precision * recall) / (precision + recall + 1e-9)
mae_basal = mean_absolute_error(y_true_basal, y_pred_basal)
mae_bolus = mean_absolute_error(y_true_bolus, y_pred_bolus)
r2_basal = r2_score(y_true_basal, y_pred_basal)

# Calculate bolus amount accuracy only for correctly detected bolus events
correct_detections = (y_pred_label == 1) & (y_true == 1)
if correct_detections.any():
    bolus_mae_correct = mean_absolute_error(
        np.array(y_true_bolus)[correct_detections],
        np.array(y_pred_bolus)[correct_detections]
    )
    bolus_count = correct_detections.sum()
else:
    bolus_mae_correct = float('nan')
    bolus_count = 0

# Confusion matrix
true_positives = ((y_pred_label == 1) & (y_true == 1)).sum()
false_positives = ((y_pred_label == 1) & (y_true == 0)).sum()
false_negatives = ((y_pred_label == 0) & (y_true == 1)).sum()
true_negatives = ((y_pred_label == 0) & (y_true == 0)).sum()

print("\n" + "="*50)
print(" INSULIN RECOMMENDER FINAL REPORT")
print("="*50)
print(f" Dataset Split: Train {len(X_train)} | Val {len(X_val)} | Test {len(X_test)}")
print(f" Optimal Threshold: {best_threshold:.2f}")
print(f" Best Epoch: {best_epoch}")
print("\n BOLUS DETECTION PERFORMANCE:")
print(f"   Recall:    {recall*100:.1f}% ({true_positives}/{true_positives + false_negatives})")
print(f"   Precision: {precision*100:.1f}% ({true_positives}/{true_positives + false_positives})")
print(f"   F1 Score:  {f1*100:.1f}%")
print(f"   False Alarms: {false_positives}")
print(f"   Missed Detections: {false_negatives}")
print(f"   True Negatives: {true_negatives}")

print("\n DOSING ACCURACY:")
print(f"   Basal Rate MAE: {mae_basal:.4f}")
print(f"   Bolus Amount MAE: {mae_bolus:.3f}")
print(f"   Bolus MAE (Correct Detections): {bolus_mae_correct:.3f} ({bolus_count} events)")
print(f"   R² (Basal): {r2_basal:.3f}")

print("\n CONFUSION MATRIX:")
print(f"               Predicted")
print(f"               No Bolus   Bolus")
print(f"Actual No Bolus  {true_negatives:4d}      {false_positives:4d}")
print(f"Actual Bolus     {false_negatives:4d}      {true_positives:4d}")

# ------------------------------
#  8. Example recommendation
# ------------------------------
def recommend_insulin(glucose, carbs, hr=80, steps=0, calories=0):
    x = np.array([[glucose, calories, hr, steps, carbs]])
    x = scaler_X.transform(x)
    seq = np.expand_dims(x.repeat(sequence_length, axis=0), axis=0)
    seq_t = torch.tensor(seq, dtype=torch.float32)

    with torch.no_grad():
        pb, po, pd = model(seq_t)
    basal = scaler_basal.inverse_transform(pb.numpy())
    bolus = scaler_bolus.inverse_transform(po.numpy())
    decision = pd.item() > best_threshold

    return {
        'basal': basal.item(),
        'bolus': bolus.item(),
        'recommend_bolus': decision,
        'confidence': pd.item()
    }

print("\n" + "="*50)
print(" RECOMMENDATION EXAMPLES")
print("="*50)

# Example 1: High glucose with carbs (should recommend bolus)
example1 = recommend_insulin(glucose=220, carbs=30)
print(f"\n Example 1 - High Glucose with Carbs:")
print(f"   Glucose: 220, Carbs: 30")
print(f"   → Basal Insulin: {example1['basal']:.3f} units")
print(f"   → Bolus Insulin: {example1['bolus']:.3f} units")
print(f"   → Bolus Recommended: {example1['recommend_bolus']} (confidence: {example1['confidence']:.2f})")

# Example 2: Normal glucose, no carbs (should NOT recommend bolus)
example2 = recommend_insulin(glucose=110, carbs=0)
print(f"\n Example 2 - Normal Glucose, No Carbs:")
print(f"   Glucose: 110, Carbs: 0")
print(f"   → Basal Insulin: {example2['basal']:.3f} units")
print(f"   → Bolus Insulin: {example2['bolus']:.3f} units")
print(f"   → Bolus Recommended: {example2['recommend_bolus']} (confidence: {example2['confidence']:.2f})")

# Example 3: Borderline case
example3 = recommend_insulin(glucose=180, carbs=15)
print(f"\n Example 3 - Borderline Case:")
print(f"   Glucose: 180, Carbs: 15")
print(f"   → Basal Insulin: {example3['basal']:.3f} units")
print(f"   → Bolus Insulin: {example3['bolus']:.3f} units")
print(f"   → Bolus Recommended: {example3['recommend_bolus']} (confidence: {example3['confidence']:.2f})")

print("\n" + "="*50)
print(" MODEL TRAINING COMPLETE!")
print("="*50)

Data range: 2018-06-13 18:40:00 → 2018-06-27 23:55:00
Created 4086 sequences
Training: 2614 sequences
Validation: 654 sequences
Test: 818 sequences
Starting training with validation...


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Epoch 10/100 - Train Loss: 0.2010 | Val F1: 0.984 | Val Recall: 0.984 | Val Precision: 0.984
Epoch 20/100 - Train Loss: 0.1784 | Val F1: 0.976 | Val Recall: 0.984 | Val Precision: 0.968
Epoch 30/100 - Train Loss: 0.1692 | Val F1: 0.984 | Val Recall: 0.984 | Val Precision: 0.984
 Early stopping at epoch 31
 Best validation F1: 0.992 at epoch 16
Loading best model for testing...

 Finding optimal threshold...
 Optimal threshold: 0.30

 INSULIN RECOMMENDER FINAL REPORT
 Dataset Split: Train 2614 | Val 654 | Test 818
 Optimal Threshold: 0.30
 Best Epoch: 16

 BOLUS DETECTION PERFORMANCE:
   Recall:    93.4% (71/76)
   Precision: 98.6% (71/72)
   F1 Score:  95.9%
   False Alarms: 1
   Missed Detections: 5
   True Negatives: 741

 DOSING ACCURACY:
   Basal Rate MAE: 0.1839
   Bolus Amount MAE: 0.016
   Bolus MAE (Correct Detections): 0.005 (71 events)
   R² (Basal): 0.087

 CONFUSION MATRIX:
               Predicted
               No Bolus   Bolus
Actual No Bolus   741         1
Actual Bolus

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
